# Dataset Translation
In this notebook, I will translate the sentences from the `b-mc2/sql-create-context` dataset into portuguese locally using the model `facebook/nllb-200-distilled-1.3B`.

In [1]:
from huggingface_hub import hf_hub_download
import pandas as pd
import os
from huggingface_hub import login
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import numpy as np
from tqdm import tqdm
from langchain.callbacks import get_openai_callback

import torch
assert torch.cuda.is_available()

from transformers import BitsAndBytesConfig


print(f"Device name: '{torch.cuda.get_device_name()}'")
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

Device name: 'NVIDIA GeForce RTX 4060 Ti'
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 4060 Ti', major=8, minor=9, total_memory=16059MB, multi_processor_count=34)'
Suporta bfloat16.


/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
class Translator:
    languages = {
        "en": "eng_Latn",
        "pt": "por_Latn",
    }

    def __init__(self, tokenizer, model, max_length=1000):
        self.tokenizer = tokenizer
        self.model = model
        self.max_length = max_length
        self._translate_to_en = pipeline(
            "translation",
            model=self.model,
            tokenizer=self.tokenizer,
            src_lang=self.languages["pt"],
            tgt_lang=self.languages["en"],
            max_length=self.max_length
        )
        self._translate_to_pt = pipeline(
            "translation",
            model=self.model,
            tokenizer=self.tokenizer,
            src_lang=self.languages["en"],
            tgt_lang=self.languages["pt"],
            max_length=self.max_length
        )

    def to_english(self, text):
        return self._translate(text, input_lang="pt", output_lang="en")

    def to_portuguese(self, text):
        return self._translate(text, input_lang="en", output_lang="pt")

    def _translate(self, text, input_lang, output_lang):
        if (input_lang=="en") and (output_lang=="pt"):
            translator = self._translate_to_pt
        elif (input_lang=="pt") and (output_lang=="en"):
            translator = self._translate_to_en
        else:
            print(f"Input and/or output languages not recognized.")
            
        output = translator(text)
        translated_text = output[0]['translation_text']
        return translated_text


# 4-bits

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

checkpoint = "facebook/nllb-200-distilled-1.3B" # "facebook/nllb-200-distilled-600M"

quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", quantization_config=quant_config)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, device_map="auto", quantization_config=quant_config)

translator_4bits = Translator(model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
translator_4bits.to_portuguese("example of text")

'exemplo de texto'

In [5]:
translator_4bits.to_english("exemplo de texto")

'example text'

### Translate SQL database

In [ ]:
update = False

filepath = "data/raw/sql_create_context_v4.parquet"

if update:
    login(token=os.environ["HUGGINGFACE_TOKEN"])
    
    REPO_ID = "b-mc2/sql-create-context"
    FILENAME = "sql_create_context_v4.json"
    
    dataset = pd.read_json(
        hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset", force_download=True)
    )
    dataset.to_parquet(filepath)

dataset = pd.read_parquet(filepath)

In [14]:
%%time

translated_path = "data/processed/translated"
n_batch = 100

n_iterations = 143

for j in range(n_iterations):
    print("interacao",j)
    try:
        batches = [int(x.replace(".parquet", "")) for x in os.listdir(translated_path)]
        new_batch = str(int(round(np.max(batches)+1))).zfill(4)
        translated = pd.read_parquet(translated_path)
        done = translated["index"].to_list()
        
        elegible = [x for x in dataset.index if x not in done]
        selected_ids = np.random.choice(elegible, n_batch)
        selected = dataset[dataset.reset_index()["index"].isin(selected_ids)]
        
        
        responses = []
        total_cost = 0
        for i, row in tqdm(selected[["question"]].iterrows()):
            query = row["question"]
            with get_openai_callback() as cb:
                # response =  chain.invoke(input={"query": query}).content
                response = translator_4bits.to_portuguese(query)
                total_cost += cb.total_cost
                responses.append(dict(index=i, translated=response))
        # print(f"total_cost = {total_cost}")
        
        translated = pd.DataFrame(responses)
        translated.to_parquet(f"{translated_path}/{new_batch}.parquet")
    except:
        pass

interacao 0


8it [00:04,  1.95it/s]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
100it [00:59,  1.68it/s]


interacao 1


100it [01:02,  1.61it/s]


interacao 2


...

interacao 141


56it [00:32,  1.72it/s]


interacao 142


18it [00:11,  1.54it/s]

CPU times: user 2h 45min 10s, sys: 6.75 s, total: 2h 45min 17s
Wall time: 2h 44min 48s
